In [120]:
#данный скрипт: 


## - осуществляет парсинг ежедневных чартов
### - через API: Deezer
### - через requests: Apple Music, Yandex
### - через selenium: VK
#### - нужен логин и пароль (а также - пока что - запуск браузера)

## - должен запускаться каждый день один раз в сутки. Самое раннее - в 11:30 утра.
## Справка: время обновления исходных чартов.

### Apple Music: 12 a.m. PST  =  10 a.m. Moscow (летом) = 11 a.m. Moscow (зимой)
#### => обновлять в 11:30 утра по Москве

### VK: 2:45 a.m Москва
### => обновлять вместе с Apple Music

### Deezer: 10 a.m. Москва
### => обновлять вместе с Apple Music 



## - на выходе:
### - обновляет уже хранящиеся данные в csv файлах каждого стриминга, лежащие в корневой директории

In [1]:
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from time import sleep
from random import randint
import datetime

In [2]:
#установка и импорт selenium
!pip install selenium
from selenium import webdriver as wb
!pip install chromedriver

In [3]:
#задаем команду для получения даты
currentDT = datetime.datetime.now() 

In [280]:
# технический код, созданный для момента, когда не было исходных файлов в директории
#for i in ["vk", "apple", "deezer", "yandex"]:
#    cols = ['rank', 'title', 'artist', "date"]
#    name = "all_" + i + ".csv" 
#    pd.DataFrame(columns = cols).to_csv(name, encoding= "utf-8")

### Deezer Russia Top 100

In [4]:
request_deezer = requests.get('https://api.deezer.com/playlist/1116189381') #ссылка на постоянный плейлист
deezer_chart_json = request_deezer.json() #через API получаем json 
deezer_top_100_daily = pd.DataFrame(deezer_chart_json['tracks']['data']) #выбираем только список треков
artists_deezer_top = [] #поскольку имена артистов "запакованы", осуществляем распаковку через цикл и заново приклеиваем к датафрейму
for i in range(0, 100):
    artists_deezer_top.append(deezer_chart_json['tracks']['data'][i]['artist']['name'])
deezer_top_100_daily['artist'] = artists_deezer_top
deezer_top_100_daily['rank'] = deezer_top_100_daily.reset_index().index +1 
deezer_top_100_daily = deezer_top_100_daily[['rank', 'title', 'artist']]
deezer_top_100_daily["date"] = currentDT.strftime("%d/%m/%Y")  

# чарт этого дня готов

In [5]:
deezer_top_100_daily

,rank,title,artist,date
0,1,Blinding Lights,The Weeknd,05/09/2020
1,2,Юность,DaBro,05/09/2020
2,3,Take Off Panties,XOLERA,05/09/2020
3,4,Roses (Imanbek Remix),SAINt JHN,05/09/2020
4,5,Сияй,Ramil',05/09/2020
...,...,...,...,...
95,96,Beverly Hills,Zivert,05/09/2020
96,97,Believer,Imagine Dragons,05/09/2020
97,98,ICE (feat. MORGENSHTERN),Morgenshtern,05/09/2020
98,99,Salt,Ava Max,05/09/2020


In [6]:
#берем имеющийся в корневой директории csv файл и обновляем его

all_deezer = pd.read_csv("all_deezer.csv")
all_deezer = all_deezer.drop(all_deezer.columns[[0]], axis=1) #удаляем получающуюся после импорта лишнюю колонку 
frames = [all_deezer, deezer_top_100_daily]
all_deezer = pd.concat(frames, sort=False)
all_deezer.to_csv("all_deezer.csv", encoding = "utf-8")

### Apple Music

In [7]:
base_url = 'https://music.apple.com/us/playlist/top-100-russia/pl.728bd30a9247487c80a483f4168a9dcd'
r = requests.get(base_url)
sleep(randint(1,3))
soup = BeautifulSoup(r.text, 'html.parser')
table = soup.findAll('div', attrs={'class':'header-and-songs-list'})
songs = soup.findAll('div', attrs={'class':'song-name typography-label'})
artists = soup.findAll('div', attrs={'class':'by-line typography-caption'})

#чистим названия треков
songs_clean = BeautifulSoup(str(songs), "lxml").text.split(", \n")
new_l=[]
for i in songs_clean:
    v = i.replace("\n", "")
    v = v.replace("[", "")
    v = v.replace("]", "")
    v = v.strip(" ")
    new_l.append(v)
songs_clean = new_l


#чистим названия артистов
artists_clean = BeautifulSoup(str(artists), "lxml").text.split("\n, \n")
new_l=[]
for i in artists_clean:
    v = i.replace("\n", "")
    v = v.replace("[", "")
    v = v.replace("]", "")
    v = v.strip(" ")
    new_l.append(v)
artists_clean = new_l

apple_music_top_100_daily = pd.DataFrame()
apple_music_top_100_daily['title'] = songs_clean
apple_music_top_100_daily['artist'] = artists_clean

apple_music_top_100_daily['rank'] = apple_music_top_100_daily.reset_index().index +1
apple_music_top_100_daily = apple_music_top_100_daily[['rank', 'title', 'artist']]
apple_music_top_100_daily["date"] = currentDT.strftime("%d/%m/%Y")  #добавляем дату


In [8]:
apple_music_top_100_daily

,rank,title,artist,date
0,1,БЕГИ (feat. Poët),DJ Smash,05/09/2020
1,2,Coco L'Eau,Egor Kreed & The Limba,05/09/2020
2,3,неболей,Basta & Zivert,05/09/2020
3,4,Lollipop,Allj & MORGENSHTERN,05/09/2020
4,5,Сияй,Ramil',05/09/2020
...,...,...,...,...
95,96,МАЛЬЧИК,ФОГЕЛЬ,05/09/2020
96,97,Midnight Sky,Miley Cyrus,05/09/2020
97,98,Зелёные волны,Zivert,05/09/2020
98,99,Пряталась в ванной,Mari Kraymbreri,05/09/2020


In [9]:
#берем имеющийся в корневой директории csv файл и обновляем его

all_apple = pd.read_csv("all_apple.csv")
all_apple = all_apple.drop(all_apple.columns[[0]], axis=1) #удаляем получающуюся после импорта лишнюю колонку 
frames = [all_apple, apple_music_top_100_daily]
all_apple = pd.concat(frames, sort=False)
all_apple.to_csv("all_apple.csv", encoding = "utf-8")

### Yandex Музыка

In [10]:
#базовая ссылка на последний актуальный чарт по России
base_url = 'https://music.yandex.ru/chart'
r = requests.get(base_url)
#на всякий случай поставим на паузу
sleep(randint(1,3))

#находим в верстке сайта интересующие нас части
soup = BeautifulSoup(r.text, 'html.parser')
songs = soup.findAll('div', attrs={'class':'d-track__name'})
artists = soup.findAll('span', attrs={'class':'d-track__artists'})

#делаем список вторичных названий песен (слов вроде remix, cover, и тд), чтобы они не сливались с названиями 
sec_titles = soup.findAll('span', attrs={'class':'d-track__version deco-typo-secondary'})
sec_titles_clean = [i.get_text() for i in sec_titles]
sec_titles_clean = sorted(sec_titles_clean, reverse=True, key=len)

#чистим названия песен и артистов
songs_clean = [i.get_text() for i in songs]
new_l=[]
for i in songs_clean:
    for j in sec_titles_clean:
        if j in i:
            v = i.replace(j, " ("+j+")")
            break
        else:
            v = i
    new_l.append(v)
songs_clean = new_l
artists_clean = [i.get_text() for i in artists]

yandex_music_top_100_daily = pd.DataFrame()
yandex_music_top_100_daily['title'] = songs_clean
yandex_music_top_100_daily['artist'] = artists_clean
yandex_music_top_100_daily['rank'] = yandex_music_top_100_daily.reset_index().index +1
yandex_music_top_100_daily= yandex_music_top_100_daily[['rank', 'title', 'artist']]
yandex_music_top_100_daily["date"] = currentDT.strftime("%d/%m/%Y")  


# чарт этого дня готов



In [11]:
yandex_music_top_100_daily

,rank,title,artist,date
0,1,Юность,Dabro,05/09/2020
1,2,Сияй,Ramil',05/09/2020
2,3,Краш,Клава Кока & NILETTO,05/09/2020
3,4,Поболело и прошло,HENSY,05/09/2020
4,5,неболей,"Zivert, Баста",05/09/2020
...,...,...,...,...
95,96,Босая,#2Маши,05/09/2020
96,97,2 типа людей,Макс Корж,05/09/2020
97,98,Brother Louie,"Vize, Imanbek, Dieter Bohlen, Leony",05/09/2020
98,99,Give It Away,"Filatov & Karas, Deepest Blue",05/09/2020


In [12]:
#берем имеющийся в корневой директории csv файл и обновляем его

all_yandex = pd.read_csv("all_yandex.csv")
all_yandex = all_yandex.drop(all_yandex.columns[[0]], axis=1) #удаляем получающуюся после импорта лишнюю колонку 
frames = [all_yandex, yandex_music_top_100_daily]
all_yandex = pd.concat(frames, sort=False)
all_yandex.to_csv("all_yandex.csv", encoding = "utf-8")

### VK 

In [13]:
#selenium-часть
url='https://vk.com'
br = wb.Chrome("/Users/sergey/chromedriver") #заменить на директорию, в которой лежит нужный драйвер 
br.get(url)
sleep(randint(2,4))
e_mail_window = br.find_element_by_css_selector("#index_email")
password_window = br.find_element_by_css_selector("#index_pass")
login_vk = "diplodok.95@mail.ru"
#"input('Введите логин: ') #изменить при дальнейшем использовании на специально зарегистрированного пользователя? 
password_vk = "aintthateasy"
#input('Введите пароль: ') #изменить при использовании (см выше)
e_mail_window.send_keys(login_vk)
password_window.send_keys(password_vk)
button1 = br.find_element_by_xpath('//*[(@id = "index_login_button")]')
button1.click()
sleep(randint(19,20))
button2 = br.find_element_by_xpath('//*[(@id = "l_aud")]//*[contains(concat( " ", @class, " " ), concat( " ", "fl_l", " " ))]')
button2.click()
sleep(randint(4,5))
button3 = br.find_element_by_css_selector('div#content li._audio_section_tab__explore > a')
button3.click()
sleep(randint(4,5))
button4 = br.find_element_by_css_selector('div#content div.CatalogBlock__recoms_top_audios_global_header.CatalogBlock__header > div > a')
button4.click()
sleep(randint(10,11))
soup = BeautifulSoup(br.page_source)
br.quit()

In [14]:
#работаем с html

songs = soup.findAll('span', attrs={'class':"audio_row__title_inner _audio_row__title_inner"})
artists = soup.findAll('div', attrs={'class':"audio_row__performers"})

songs_clean = [i.get_text() for i in songs]
artists_clean = [i.get_text() for i in artists]

data = {"rank": [i for i in range(1, 101)], "title": songs_clean, "artist":artists_clean}
vk_music_top_100_daily = pd.DataFrame(data)
vk_music_top_100_daily["date"] = currentDT.strftime("%d/%m/%Y")  


In [15]:
vk_music_top_100_daily

,rank,title,artist,date
0,1,Coco L'Eau,"Егор Крид, The Limba",05/09/2020
1,2,Lollipop,"Элджей, MORGENSHTERN",05/09/2020
2,3,ERRDAY,Big Baby Tape,05/09/2020
3,4,Вали,Канги,05/09/2020
4,5,Если тебе будет грустно,"Rauf & Faik, NILETTO",05/09/2020
...,...,...,...,...
95,96,Плачу на техно,"Cream Soda, ХЛЕБ",05/09/2020
96,97,Пикачу,"MIA BOYKA, ЕГОР ШИП",05/09/2020
97,98,Tantra,Miyagi & Andy Panda,05/09/2020
98,99,Море волнуется раз,Konfuz,05/09/2020


In [16]:
#берем имеющийся в корневой директории csv файл и обновляем его

all_vk = pd.read_csv("all_vk.csv")
all_vk = all_vk.drop(all_vk.columns[[0]], axis=1) #удаляем получающуюся после импорта лишнюю колонку 
frames = [all_vk, vk_music_top_100_daily]
all_vk = pd.concat(frames, sort=False)
all_vk.to_csv("all_vk.csv", encoding = "utf-8")